In [4]:
import wandb
with wandb.init() as run: 
  artifact = run.use_artifact(
  #'aaac/aaac_model_runs/aaac_raw:v2', 
  "aaac/dataset_versions/aaac_multi_angle:v14",
  type='dataset'
  )

artifact_dir = artifact.download()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aaac (use `wandb login --relogin` to force relogin)


In [5]:
from datasets import load_dataset
import os
#dataset = load_dataset('json', data_files=os.path.join(artifact_dir,"aaac.jsonl"))
dataset = load_dataset('json', data_files=os.path.join(artifact_dir,"pre_aaac-20210427_informal_no-lmpara.jsonl"))

dataset

Using custom data configuration default-9ab1507cf0173569
Reusing dataset json (/home/kit/itz/kz1174/.cache/huggingface/datasets/json/default-9ab1507cf0173569/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02)


DatasetDict({
    train: Dataset({
        features: ['argument_source', 'argdown_reconstruction', 'reason_statements', 'conclusion_statements', 'premises', 'premises_formalized', 'conclusion', 'conclusion_formalized', 'intermediary_conclusions_formalized', 'intermediary_conclusions', 'distractors', 'id', 'predicate_placeholders', 'entity_placeholders', 'steps', 'n_premises', 'n_distractors', 'base_scheme_groups', 'scheme_variants', 'domain_id', 'domain_type', 'plcd_subs', 'argdown_index_map', 'presentation_parameters'],
        num_rows: 10000
    })
})

In [50]:
filtered_dataset = dataset.filter(lambda example: not "complex variant" in example['scheme_variants'])
filtered_dataset = dataset #no filter

In [81]:
filtered_dataset

DatasetDict({
    train: Dataset({
        features: ['argument_source', 'argdown_reconstruction', 'reason_statements', 'conclusion_statements', 'premises', 'premises_formalized', 'conclusion', 'conclusion_formalized', 'intermediary_conclusions_formalized', 'intermediary_conclusions', 'distractors', 'id', 'predicate_placeholders', 'entity_placeholders', 'steps', 'n_premises', 'n_distractors', 'base_scheme_groups', 'scheme_variants', 'domain_id', 'domain_type', 'plcd_subs', 'argdown_index_map', 'presentation_parameters'],
        num_rows: 10000
    })
})

In [65]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples,source_additional,target):
    
    source_additional_list = []
    if source_additional == "":
        source_additional_list = [""]*len(examples["argument_source"])
    else:
        for instance in examples[source_additional]:
            if not (source_additional=="argdown_reconstruction"):
                text = ""
                for j,element in enumerate(instance):
                    text+= " " + element["text"] + " {ref: (" + str(element["ref_reco"]) + ")} |"
                source_additional_list.append(" " + source_additional + ":" + text)
            else:
                source_additional_list.append(" " + source_additional + ": " + instance)
                
    for i,element in enumerate(source_additional_list):
        source_additional_list[i]+=" " + target + ":"
        
    
    target_list = []
    for instance in examples[target]:
        if not (target == "argdown_reconstruction"):
            text = ""
            for j,element in enumerate(instance):
                text+= " " + element["text"] + " {ref: (" + str(element["ref_reco"]) + ")} |"
            target_list.append(text)
        else:
            target_list.append(instance) 
    

    inputs = ["argument_source: " + instance + source_additional_list[i] for i,instance in enumerate(examples["argument_source"])]
    #model_inputs = tokenizer(inputs, truncation=True)

    # Setup the tokenizer for targets
    
    #with tokenizer.as_target_tokenizer():
    #    labels = tokenizer(target_list,  truncation=True)

    argdown = {}
    argdown["id"] = examples["id"]
    #argdown["question"] = {}
    argdown["question"] = inputs ##<-- input 
    argdown["output"] = target_list##<-- output 
    argdown["prefix"] = ["gen:"] * len(target_list) ##<-- important, indicates that we will measure accuracy
    return argdown


In [48]:

def preprocess_function_no_source(examples):
    
    reason_statements_list = []

    for instance in examples["reason_statements"]:
        text = ""
        for j,element in enumerate(instance):
            text+= " " + element["text"] + " {ref: (" + str(element["ref_reco"]) + ")} |"
        reason_statements_list.append("reason_statements" + ":" + text)
    
    conclusion_statements_list = []

    for instance in examples["conclusion_statements"]:
        text = ""
        for j,element in enumerate(instance):
            text+= " " + element["text"] + " {ref: (" + str(element["ref_reco"]) + ")} |"
        conclusion_statements_list.append("conclusion_statements" + ":" + text)

                
                     
    for i,element in enumerate(conclusion_statements_list):
        conclusion_statements_list[i]+= " " + "argdown_reconstruction" + ":"
        
    
    target_list = []
    for instance in examples["argdown_reconstruction"]:
        target_list.append(instance) 
    

    inputs = [reason_statements_list[i] + " " + conclusion_statements_list[i] for i,instance in enumerate(reason_statements_list)]
    #model_inputs = tokenizer(inputs, truncation=True)

    # Setup the tokenizer for targets
    
    #with tokenizer.as_target_tokenizer():
    #    labels = tokenizer(target_list,  truncation=True)

    argdown = {}
    argdown["id"] = examples["id"]
    #argdown["question"] = {}
    argdown["question"]= inputs ##<-- input 
    argdown["output"] = target_list##<-- output 
    argdown["prefix"] = ["gen:"] * len(target_list) ##<-- important, indicates that we will measure accuracy
    return argdown


In [82]:
from datasets import concatenate_datasets, load_dataset


processed_dataset = concatenate_datasets([filtered_dataset.map(lambda example: preprocess_function(example,"","argdown_reconstruction"),batched=True)["train"],
                                         filtered_dataset.map(lambda example: preprocess_function(example,"reason_statements","argdown_reconstruction"),batched=True)["train"],
                                         filtered_dataset.map(lambda example: preprocess_function(example,"conclusion_statements","argdown_reconstruction"),batched=True)["train"],
                                         filtered_dataset.map(lambda example: preprocess_function(example,"","reason_statements"),batched=True)["train"],
                                         filtered_dataset.map(lambda example: preprocess_function(example,"argdown_reconstruction","reason_statements"),batched=True)["train"],
                                         filtered_dataset.map(lambda example: preprocess_function(example,"conclusion_statements","reason_statements"),batched=True)["train"],
                                         filtered_dataset.map(lambda example: preprocess_function(example,"","conclusion_statements"),batched=True)["train"],
                                         filtered_dataset.map(lambda example: preprocess_function(example,"argdown_reconstruction","conclusion_statements"),batched=True)["train"],
                                         filtered_dataset.map(lambda example: preprocess_function(example,"reason_statements","conclusion_statements"),batched=True)["train"],
                                        filtered_dataset.map(lambda example: preprocess_function_no_source(example),batched=True)["train"]])

In [52]:
processed_dataset

Dataset({
    features: ['argdown_index_map', 'argdown_reconstruction', 'argument_source', 'base_scheme_groups', 'conclusion', 'conclusion_formalized', 'conclusion_statements', 'distractors', 'domain_id', 'domain_type', 'entity_placeholders', 'id', 'intermediary_conclusions', 'intermediary_conclusions_formalized', 'n_distractors', 'n_premises', 'output', 'plcd_subs', 'predicate_placeholders', 'prefix', 'premises', 'premises_formalized', 'presentation_parameters', 'question', 'reason_statements', 'scheme_variants', 'steps'],
    num_rows: 100000
})

In [83]:
processed_dataset = processed_dataset.map(remove_columns=['argdown_index_map', 'argdown_reconstruction', 'argument_source', 'base_scheme_groups', 'conclusion', 'conclusion_formalized', 'conclusion_statements', 'distractors', 'domain_id', 'domain_type', 'entity_placeholders',  'intermediary_conclusions', 'intermediary_conclusions_formalized', 'n_distractors', 'n_premises',  'plcd_subs', 'predicate_placeholders', 'premises', 'premises_formalized', 'presentation_parameters', 'reason_statements', 'scheme_variants', 'steps'])
processed_dataset

Dataset({
    features: ['id', 'output', 'prefix', 'question'],
    num_rows: 100000
})

In [84]:
processed_dataset = processed_dataset.train_test_split(test_size=0.1)
processed_dataset["dev"] = processed_dataset["test"].shard(num_shards=2,index=0)
processed_dataset["test"] = processed_dataset["test"].shard(num_shards=2,index=1)
processed_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'output', 'prefix', 'question'],
        num_rows: 90000
    })
    test: Dataset({
        features: ['id', 'output', 'prefix', 'question'],
        num_rows: 5000
    })
    dev: Dataset({
        features: ['id', 'output', 'prefix', 'question'],
        num_rows: 5000
    })
})

In [94]:
import torch
dataloader_train= torch.utils.data.DataLoader(processed_dataset["train"], batch_size=1,shuffle=False)
dataloader_test= torch.utils.data.DataLoader(processed_dataset["test"], batch_size=1,shuffle=False)
dataloader_dev= torch.utils.data.DataLoader(processed_dataset["dev"], batch_size=1,shuffle=False)

In [101]:
import tempfile
import json
from tqdm import tqdm_notebook
run = wandb.init(entity="aaac",project="dataset_versions",name="concat_dataset")

with tempfile.TemporaryDirectory() as tempdir: 
    file_out = os.path.join(tempdir,"train"+".jsonl")
    write_file = open(file_out,'w')
    for element in tqdm_notebook(processed_dataset["train"]):
        element["question"] = {"stem": element["question"] }
        write_file.write(json.dumps(element))
        write_file.write("\n")
    write_file.close()
    
    file_out = os.path.join(tempdir,"test"+".jsonl")
    write_file = open(file_out,'w')
    for element in tqdm_notebook(processed_dataset["test"]):
        element["question"] = {"stem": element["question"] }
        write_file.write(json.dumps(element))
        write_file.write("\n")
    write_file.close()
    
    file_out = os.path.join(tempdir,"dev"+".jsonl")
    write_file = open(file_out,'w')
    for element in tqdm_notebook(processed_dataset["dev"]):
        element["question"] = {"stem": element["question"] }
        write_file.write(json.dumps(element))
        write_file.write("\n")
    write_file.close()
    
    artifact = wandb.Artifact("concat_dataset",type='dataset')
    artifact.add_dir(tempdir)
    run.log_artifact(artifact)
    run.finish()
    
    

Error in callback <function _WandbInit._resume_backend at 0x14e9a97556a8> (for pre_run_cell):


Exception: The wandb backend process has shutdown

wandb: ERROR Problem finishing run
Traceback (most recent call last):
  File "/home/kit/itz/kz1174/.local/lib/python3.6/site-packages/wandb/sdk/wandb_run.py", line 1502, in _atexit_cleanup
    self._on_finish()
  File "/home/kit/itz/kz1174/.local/lib/python3.6/site-packages/wandb/sdk/wandb_run.py", line 1659, in _on_finish
    self._backend.interface.publish_telemetry(self._telemetry_obj)
  File "/home/kit/itz/kz1174/.local/lib/python3.6/site-packages/wandb/sdk/interface/interface.py", line 231, in publish_telemetry
    self._publish(rec)
  File "/home/kit/itz/kz1174/.local/lib/python3.6/site-packages/wandb/sdk/interface/interface.py", line 524, in _publish
    raise Exception("The wandb backend process has shutdown")
Exception: The wandb backend process has shutdown


/opt/bwhpc/common/jupyter/base/lib/python3.6/site-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


/opt/bwhpc/common/jupyter/base/lib/python3.6/site-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/opt/bwhpc/common/jupyter/base/lib/python3.6/site-packages/ipykernel_launcher.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


wandb: Adding directory to artifact (/scratch/slurm_tmpdir/job_19503324/tmpmjs8g9f3)... 

Done. 1.1s
